# **Obtención del conjunto de datos a través de CoinMarket**

In [4]:
import requests
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.io as pio


# Configura tu clave de API
api_key = '74ed5d5c-85df-44bb-8e8a-7751e2f92b0f'  # Reemplaza con tu clave de API de CoinMarketCap

In [8]:
# Obtener datos de CoinMarketCap
def get_cryptos_data_csv(api_key):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    params = {
        'start': '1',
        'limit': '1000',  # Ajusta el límite según sea necesario
        'convert': 'USD'
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data['data']


# Extraer atributos y guardar en CSV
def save_cryptos_to_csv(cryptos):
    # Crear una lista de diccionarios con todos los atributos
    records = []
    for crypto in cryptos:
        # Check if 'platform' key exists and has a value before accessing 'name'
        platform = crypto.get('platform')
        if platform:
            platform_name = platform.get('name', 'N/A')
        else:
            platform_name = 'N/A'

        record = {
            'id': crypto['id'],
            'name': crypto['name'],
            'symbol': crypto['symbol'],
            'slug': crypto['slug'],
            'num_market_pairs': crypto['num_market_pairs'],
            'date_added': crypto['date_added'],
            'tags': ', '.join(crypto.get('tags', [])),
            'max_supply': crypto.get('max_supply'),
            'circulating_supply': crypto['circulating_supply'],
            'total_supply': crypto['total_supply'],
            'platform': platform_name, # Use platform_name variable
            'quote': crypto['quote']['USD']
        }
        records.append(record)

    # Convertir a DataFrame y guardar en CSV
    df = pd.DataFrame(records)
    df.to_csv('cryptos_data_full.csv', index=False)  # Guarda la tabla como archivo CSV
    print("Archivo CSV guardado como 'cryptos_data_full.csv'")

# Obtener datos y guardar en CSV
cryptos_data = get_cryptos_data_csv(api_key)
save_cryptos_to_csv(cryptos_data)

Archivo CSV guardado como 'cryptos_data_full.csv'


# **Despliege del modelo**

In [6]:
# Obtener datos de CoinMarketCap
def get_cryptos_data(api_key):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    params = {
        'start': '1',
        'limit': '1000',  # Ajusta el límite según sea necesario
        'convert': 'USD'
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data['data']

# Extraer criptomonedas y sus tipos
def extract_cryptos_and_types(cryptos):
    type_dict = {}
    volume_dict = {}
    price_dict = {}
    id_dict = {}
    global_ranking_dict = {}
    local_ranking_dict = {}

    # Categorizar criptomonedas por tipo y almacenar volumen, precio, ID y ranking global
    for crypto in cryptos:
        name = crypto['name']
        symbol = crypto['symbol']
        volume_24h = crypto['quote']['USD']['volume_24h']  # Obtener volumen de 24h
        price = crypto['quote']['USD']['price']  # Obtener precio actual
        crypto_id = crypto['id']  # Obtener ID de la criptomoneda
        global_rank = crypto['cmc_rank']  # Obtener ranking global

        # Obtener el tipo de criptomoneda
        tags = crypto.get('tags', [])
        if tags:
            type_ = tags[0]  # Usar el primer tag como el tipo
        else:
            type_ = 'Desconocido'

        if type_ not in type_dict:
            type_dict[type_] = []

        type_dict[type_] .append(name)
        volume_dict[name] = volume_24h
        price_dict[name] = price
        id_dict[name] = crypto_id  # Guardar ID
        global_ranking_dict[name] = global_rank

    # Calcular el ranking dentro de cada tipo
    for type_, names in type_dict.items():
        sorted_names = sorted(names, key=lambda n: volume_dict.get(n, 0), reverse=True)
        for rank, name in enumerate(sorted_names, start=1):
            local_ranking_dict[name] = rank

    # Filtrar tipos con pocas criptomonedas
    filtered_type_dict = {k: v for k, v in type_dict.items() if len(v) > 1}
    return filtered_type_dict, volume_dict, price_dict, id_dict, global_ranking_dict, local_ranking_dict

# Crear el grafo bipartito
def create_bipartite_graph(types):
    B = nx.Graph()

    # Añadir nodos de criptomonedas y tipos
    cryptos = [name for names in types.values() for name in names]
    B.add_nodes_from(cryptos, bipartite=0)  # Nodo criptomonedas
    B.add_nodes_from(types.keys(), bipartite=1)  # Nodo tipos

    # Añadir aristas entre criptomonedas y tipos
    for type_, crypto_names in types.items():
        for name in crypto_names:
            B.add_edge(name, type_)

    return B

# Visualizar el grafo bipartito con etiquetas de ranking
def visualize_bipartite_graph_plotly(B, volume_dict, price_dict, id_dict, global_ranking_dict, local_ranking_dict):
    pos = nx.multipartite_layout(B, subset_key="bipartite", align="horizontal")
    pos = nx.spring_layout(B, pos=pos, iterations=200, k=0.3)  # Ajuste para que los nodos estén más unidos

    edge_x = []
    edge_y = []
    edge_labels_x = []
    edge_labels_y = []
    edge_labels_text = []

    for edge in B.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_y.append(y0)
        edge_y.append(y1)

        # Calcular la posición media de la arista para mostrar la etiqueta
        edge_labels_x.append((x0 + x1) / 2)
        edge_labels_y.append((y0 + y1) / 2)

        # Añadir la etiqueta con el ranking local
        if edge[0] in local_ranking_dict:
            rank = local_ranking_dict[edge[0]]  # Buscar el ranking local de la criptomoneda dentro de su tipo
            edge_labels_text.append(f'Rank: {rank}')  # Mostrar el ranking local
        else:
            edge_labels_text.append('')

    node_x = []
    node_y = []
    node_sizes = []
    node_colors = []
    node_text = []  # Para el texto en hover

    # Normalizar el volumen para el tamaño de los nodos
    volumes = np.array(list(volume_dict.values()))
    if len(volumes) > 0:
        volume_log = np.log1p(volumes)  # Usar log1p para normalizar
        min_vol, max_vol = min(volume_log), max(volume_log)
        volume_normalized = (volume_log - min_vol) / (max_vol - min_vol)
    else:
        volume_normalized = np.zeros(len(volume_dict))

    for node, (x, y) in pos.items():
        node_x.append(x)
        node_y.append(y)
        if B.nodes[node].get('bipartite') == 0:
            # Tamaño de los nodos basado en el volumen de transacciones
            size = 10 + 40 * volume_normalized[list(volume_dict.keys()).index(node)]  # Tamaño de nodo ajustado
            color = f'rgba(0, 0, 255, {volume_normalized[list(volume_dict.keys()).index(node)]})'  # Color basado en volumen
        else:
            size = 10
            color = 'green'

        node_sizes.append(size)
        node_colors.append(color)

        # Configurar el texto para el hover
        if B.nodes[node].get('bipartite') == 0:
            text = f'{node}<br>ID: {id_dict.get(node, "N/A")}<br>Price: ${price_dict.get(node, "N/A"):,.2f}<br>Volume: ${volume_dict.get(node, "N/A"):,.2f}<br>Global Rank: {global_ranking_dict.get(node, "N/A")}'
        else:
            text = node
        node_text.append(text)

    fig = go.Figure()

    # Añadir aristas
    fig.add_trace(go.Scatter(
        x=edge_x,
        y=edge_y,
        mode='lines',
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',  # Deshabilitar hoverinfo en las aristas
        name='Local Ranking (by types)'  # Nombre actualizado para la leyenda
    ))

    # Añadir etiquetas de ranking local a las aristas
    for i in range(len(edge_labels_text)):
        fig.add_trace(go.Scatter(
            x=[edge_labels_x[i]],
            y=[edge_labels_y[i]],
            mode='text',
            text=[edge_labels_text[i]],
            textfont=dict(size=10, color='#000'),
            showlegend=False
        ))

    # Añadir nodos de criptomonedas
    fig.add_trace(go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=[n.split('<br>')[0] for n in node_text],  # Solo el nombre
        textposition='top center',
        marker=dict(size=node_sizes, color=node_colors, line=dict(width=2, color='DarkSlateGrey')),
        hoverinfo='text',
        hovertext=node_text,  # Mostrar ID, precio, volumen y ranking global en el hover
        name='Cryptocurrencies'  # Nombre para la leyenda
    ))

    # Añadir nodos de tipos
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(color='green', size=10, line=dict(width=2, color='DarkSlateGrey')),
        legendgroup='types',
        showlegend=True,
        name='Types',
        hoverinfo='none'
    ))

    # Añadir la leyenda de colores
    fig.update_layout(
        showlegend=True,
        legend_title='Legend',
        legend=dict(
            x=1.05,
            y=0.5,
            traceorder='normal',
            orientation='v'
        ),
        title='Grafo Bipartito: Criptomonedas y Tipos con Ranking Local y Precio',
        title_x=0.5,
        width=1200,  # Aumentar el ancho
        height=800,  # Ajustar la altura para mantener proporción
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False)
    )
    pio.write_html(fig, 'bipartite_graph.html')  # Cambia el nombre del archivo según sea necesario

    fig.show()

# Obtener datos y construir el grafo
cryptos_data = get_cryptos_data(api_key)
type_dict, volume_dict, price_dict, id_dict, global_ranking_dict, local_ranking_dict = extract_cryptos_and_types(cryptos_data)
B = create_bipartite_graph(type_dict)
visualize_bipartite_graph_plotly(B, volume_dict, price_dict, id_dict, global_ranking_dict, local_ranking_dict)


# **Busqueda de los tags totales pertenecientes a una criptodivisa**

In [7]:
# Buscar una criptomoneda por nombre o símbolo
def find_crypto_category(cryptos, name_or_symbol):
    for crypto in cryptos:
        if crypto['name'].lower() == name_or_symbol.lower() or crypto['symbol'].lower() == name_or_symbol.lower():
            name = crypto['name']
            symbol = crypto['symbol']
            tags = crypto.get('tags', [])
            return name, symbol, tags
    return None

# Obtener datos
cryptos_data = get_cryptos_data(api_key)

# Buscar Worldcoin (puedes buscar por nombre o símbolo)
name, symbol, tags = find_crypto_category(cryptos_data, "Worldcoin")

if tags:
    print(f"La criptomoneda {name} ({symbol}) pertenece a las siguientes categorías: {tags}")
else:
    print(f"No se encontraron categorías para la criptomoneda {name} ({symbol})")

La criptomoneda Worldcoin (WLD) pertenece a las siguientes categorías: ['privacy', 'zero-knowledge-proofs', 'identity', 'ethereum-ecosystem', 'governance', 'blockchain-capital-portfolio', 'a16z-portfolio', 'token', 'optimism-ecosystem']
